# PROBABILITY MATCHING, PART 2

### Example: probability matching in groups of foraging ducks

In the Gallistel reading, we see how probability matching behavior could be 'explained' / 'predicted' by a set of differential equations. Now we could create code to graph the function(s) described by those equations, but I think many of us would still find the connection between the equations and predicted behavior a bit murky. Let's instead approximate the equations with a simple loop-based simulation.

Here I have developed a streamlined / simplified version of the 'computational model' described by Gallistel. Again, we are going to generate predictions by looping over a number of time steps. This is an *iterative* approach, where we actually do calculations at each time step. For those of us who are not mathematicians or physicists, the iterative approach will likely be much more intuitive than a system of differential equations (as in the Gallistel readings). (If you are so inclined, you could try to implement a more faithful version of Gallistel's model.) 

We will specify a number of parameters, including the **rates** and **magnitudes** for the two food sources. We will assume that each ducks observes the disbursement of food and updates its estimates of prevalence at each food source. We will be interested in what proportion of ducks approach each source over time. We need to operationalize how to decide if a duck is 'at' a particular source. For our current purposes, let's simply count a duck as being 'at' a source if it is closer to that source than the other. However, we will not try to simulate precise positions of each duck; note that this level of detail is not present in Gallistel's equations. So we will simply assume that a duck is nearer the source it has selected as soon as it makes that selection. 

However, we are also not going to simulate the decision of each duck. We are going to approximate this by tracking prevalence of food at each source over time, and using a 'switchiness' constant like Gallistel and earlier researchers use in their equations. We could make this a random variable with a particular distribution -- if we were simulating the behavior of each duck. Instead, we are just going to make it a constant that scales the distribution based on prevalence. Let's walk through how we will do this.

***

1. **Initialization**: Given two food sources, Source 1 and Source 2, each source has:
   - A rate of food disbursement: $ rate_1 $ and $ rate_2 $.
   - A magnitude of the food morsel: $ magnitude_1 $ and $ magnitude_2 $.


To be very specific: 

- *$ rate_1 $*: How often Source 1 distributes food (if $ rate_1 $ = 2, this means Source 1 distributes food every second step [steps 2, 4, 6...]; so higher values are actually *slower* rates [e.g., if set to 4, food would disperse at time steps 4, 8, 12...]).
- *$ rate_2 $*: How often Source 2 distributes food
- *$ magnitude_1 $*: The magnitude of food morsels from Source 1
- *$ magnitude_2 $*: The magnitude of food morsels from Source 2
- *timesteps*: How many timesteps our predictions should include
- *switchiness*: a constant that governs how willing or reluctant ducks are, on average, to change sources. A high value would make all ducks very responsive to changes in prevalence

Then we will run a loop for a number of steps equal to *timesteps*. Source 1 will distribute food of size *$ magnitude_1 $* every *$ rate_1 $* timesteps. Source 2 will distribute food of size *$ magnitude_1 $* every *$ rate_2 $* timesteps. When this happens, estimates of prevalence will be updated. The way we do this is to track the total magnitude of food distributed at a source and divide that by the number of timesteps.

At each time step $ t $, the prevalence of food at each source is updated based on whether food is disbursed and the magnitude of that food. **Initial** prevalences (at time 0, i.e., the point where we bgin) are estimated using values we will specify for the 'history' of each source, as though the ducks have already had a chance to observe the sources for a while. For each source, we will specify a baseline *base_magnitude* (assume this much food has appeared at the source...) and *base_time* (...over this many time steps). This may seem unnecessarily complicated, but we can set these to zero to leave them out. What this *does* allow is, e.g., to run a simulation of what would happen if the ducks had been exposed to particular prevalences of food for some amount of time and now we are going to change those. So, to calculate the *initial prevalences*:

$$
prevalence_1[0] = \frac{baseMagnitude_1}{baseTime_1}
$$

$$
prevalence_2[0] = \frac{baseMagnitude_2}{baseTime_2}
$$

Ducks are initialized such that half are near each source (this is a simplification; we could consider instead making the ratio match the expected ratio given the initial prevalences):

$$
duckDistribution_1[0] = 0.5
$$

$$
duckDistribution_2[0] = 0.5
$$

2. **Prevalence Calculation**:
For each time step $ t $, if food is disbursed from a source, we update the cummulative magnitude for that source (*summed_magnitude*), and then update prevalence by dividing by time steps. For example, for Source 1 at time $t$:

$$
summedMagnitude_1 = summedMagnitude_1 + magnitude_1
$$

$$
prevalence_1[t] = \frac{summedMagnitude_1}{timeSteps_1}
$$

Similarly, for Source 2, if the current time step is divisible by $rate_2$:

$$
summedMagnitude_2 = summedMagnitude_2 + magnitude_2
$$

$$
prevalence_2[t] = \frac{summedMagnitude_2}{timeSteps_2}
$$

If no food is disbursed at a time step, the prevalence remains the same as the previous step. Note that this may be a questionable detail. Maybe we should just update prevalence at every time step? 

3. **Duck Distribution Calculation**:

If all ducks are calculating the same prevalence values, how can we simulate probability matching? Wouldn't they all just choose the denser source? Gallistel (like his predecessors in this domain) had to account for this. The solution is to propose that animals vary in their "switchiness" -- how likely they are to change sources based on prevalence. Animals are assumed to vary in this parameter, such that some are more likely to change sources, while others are more likely to stay at their current food source even if they have calculated that the other source has higher prevalence. We could assume that every duck has its own switchiness parameter. To assign values, we would set the mean, standard deviation, and distribution type (e.g., Gaussian) for switchiness. For now, let's do something much simpler. Let's assume a global switchiness value that represents the mean tendency to switch, and we will call this switchiness parameter $ s $.

Now to approximate having a distribution of values, we are going to introduce a nonlinearity. We are going to calcuate weighted prevalences at each source by exponentiating prevalence at a source multiplied by switchiness:

$$
weight_1 = \exp(s \times prevalence_1[t])
$$

$$
weight_2 = \exp(s \times prevalence_2[t])
$$

The proportion of ducks at each source is then:


$$
duckDistribution_1[t] = \frac{weight_1}{weight_1 + weight_2}
$$

$$
duckDistribution_2[t] = \frac{weight_2}{weight_1 + weight_2}
$$

***

**An aside:** for *cognitive scientists and neuroscientists*, do the calculations above remind you of anything? This is actually a common variant of the Luce (1959) choice rule. This is often expressed along the following lines. For some set of possible alternatives (e.g., words in the lexicon), we somehow calculate the evidence in support of each response alternative. We then calculate the strength of each possible response:

$$
R_i = \exp(k \times evidence_i)
$$

Note that $k$ is a constant that often has to be determined for a given task. (In general, $k$ will need to be fairly small when there are few choices and fairly large when there are many choices. For the probability matching context, $switchiness$ takes the place of $k$. Note that $k$ is normally an integer, but we are going to use a probability [ranging from 0 to 1] for switchiness below.) We then calculate the response *probability* for each response by normalizing by the sum of all response strengths. So for each alternative $i$ from $n$ possible alternatives:

$$
P_i = \frac{R_i}{\sum_{j=1}^{n} R_j}
$$

Luce and a veritable mountain of subsequent research have confirmed that this simple rule does an excellent job of approximating the behavior of humans and other animals in an incredible range of choice situations. We can discuss in class. 

However, we can also help link this to related concepts in machine learning by putting all the parts into one equation. To make it more readable, let's replace ${evidence}$ with $z$ and drop the $\times$ operators:

$$
P_i = \frac{e^{k {z}_i}}{\sum_{j=1}^{n} e^{k {z}_j}}
$$

This is just what we have done above for our two sources, where *prevalences* are the evidence and *weights* are the response strengths. 

Now, what is the link to machine learning? For *neural network modelers*, does this remind you of anything else? Note how similar it is to the *softmax* transformation:

$$
\sigma({z}_i) = \frac{e^{z_i}}{\sum_{j=1}^{n} e^{z_j}}
$$

Where ${z}_i$ is the activation value for node $i$. The only difference is that there is not typically a scaling constant in the exponentiation step (i.e., the $k$ from the previous equation is absent).

***


4. **Expected and Actual Ratios**:

The theoretically expected ratio of ducks at Source 1 based on food magnitudes and rates is very simple. We will express it as the ratio of ducks expected at Source 1. Overall prevalence at each source is just $magnitude/rate$ (that is, amount of food per unit of time). So a source with $rate$ = 2 and $magnitude$ = 4 has a prevalence of $4/2 = 2$ (that is, 2 units of food per unit of time). Then we just need the ratio of prevalence at the two sources to determine what proportion of ducks should be at Source 1 if the ducks, as a group, are probability matching. Mathematically:

$$
expectedRatio = \frac{magnitude_1/rate_1}{\left(magnitude_1/rate_1\right) + \left(magnitude_2/rate_2\right)}
$$

The actual ratio is taken from the final value of the duck distribution:

$$
actualRatio = duckDistribution_1[end]
$$

The discrepancy between the desired and actual ratios is:

$$
discrepancy = |actualRatio - expectedRatio|
$$

***

### An implementation
Now let's look at a way to implement these ideas. Below, you will find python code for the simulation. Below that code, you will find sliders you can use to adjust parameters and graphs that show the results. Play around with it a bit, and then look at the lab report section below the graphs. 

The code has many comments in it. Comments begin with `#`; anything after `#` on a line is ignored by the python interpreter. Comments can also begin and end with `'''`.

If you are new to programming and/or Python, you may not understand much of the code. That's okay. We can discuss it in class. 

In [1]:

# First, we import all the things we need

import numpy as np # numeric tools
import plotly.graph_objs as go # special features for graphing 
from ipywidgets import interactive, IntSlider, FloatSlider, HBox, VBox, Label, Button, Output, Layout # tools for interaction
from IPython.display import display # we need this to display within the notebook environment

# Define the simulation procedure, which will also serve as an 
# 'objective function' for finding the optimal value for the 
# switchniness variable. The first line specifies the variables 
# that the function expects. If no default value is specified, then 
# either the variable has to be 'global' (a poor programming practice) 
# or it has to be specified. The list of variables after def simulation(...
# have to be supplied to the function. Look below to see where it is
# 'called'.
def simulation(switchiness, rate_1, rate_2, magnitude_1, magnitude_2, summed_magnitude_1, summed_magnitude_2, timeSteps_1, timeSteps_2, timeSteps):

    # Initialize the prevalences and duck distributions 
    # by filling them with zeros
    prevalence_1 = np.zeros(timeSteps) # an array of timeSteps 0s
    prevalence_2 = np.zeros(timeSteps)
    duckDistribution_1 = np.zeros(timeSteps)
    duckDistribution_2 = np.zeros(timeSteps)

    # Start with the initial prevalences and duck distributions; note that 
    # summed_magnitude_1 and timeSteps_1 are variables we can send to the 
    # function. They serve to set prior probabiities for prevalence. That is, 
    # we use them to set a baseline assumption about how prevalent the ducks 
    # 'assume' food is at each source (a scenario where we would typically 
    # assume 50/50 in the absence of other information). But we can also use 
    # these to look at how quickly the ducks would adjust if the prevalences 
    # changed at some point. For example, we could set the prior to be a ratio
    # of 2:1. If we set the previous time steps to large values, it will take 
    # longer for a new ratio to 'settle', because we will accumulate new info
    # via these variables (see below).
    prevalence_1[0] = summed_magnitude_1 / timeSteps_1 # food per time
    prevalence_2[0] = summed_magnitude_2 / timeSteps_2
    duckDistribution_1[0] = 0.5 # we should calculate this rather than set it
    duckDistribution_2[0] = 0.5

    # Compute the prevalences and duck distributions over time
    for t in range(1, timeSteps):
        # Update the prevalences and time steps if a source disburses food
        #    Note: The '%' (modulus) operator checks whether t is divisible by 
        #    rate_1 (or rate_2) by seeing if there is a remainder if you divide 
        #    (no remainder means divisible by). 
        if t % rate_1 == 0:
            summed_magnitude_1 += magnitude_1
            #prevalence_1[t] = summed_magnitude_1 / timeSteps_1
        else:
            # If no food, we keep our old prevalence; an alternative would be to 
            # just recalculate prevalence whether food is disbursed or not
            prevalence_1[t] = prevalence_1[t-1]
        
        prevalence_1[t] = summed_magnitude_1 / timeSteps_1
            
        if t % rate_2 == 0:
            summed_magnitude_2 += magnitude_2
            #prevalence_2[t] = summed_magnitude_2 / timeSteps_2
        else:
            # If no food, we keep our old prevalence; an alternative would be to 
            # just recalculate prevalence whether food is disbursed or not
            prevalence_2[t] = prevalence_2[t-1]
            
        prevalence_2[t] = summed_magnitude_2 / timeSteps_2

        # increment both time steps
        timeSteps_1 += 1
        timeSteps_2 += 1

        # Compute the unnormalized weights for each source by scaling (nonlinearly) 
        # by switchiness
        weight_1 = np.exp(switchiness * prevalence_1[t])
        weight_2 = np.exp(switchiness * prevalence_2[t])
        #   --> Note - to see the impact of the exponential function, uncomment 
        #       the next two lines and comment out the previous 2 lines
        #weight_1 = (switchiness * prevalence_1[t])
        #weight_2 = (switchiness * prevalence_2[t])

        # Normalize the weights (making them like probabilities) to get the duck distributions
        duckDistribution_1[t] = weight_1 / (weight_1 + weight_2)
        duckDistribution_2[t] = weight_2 / (weight_1 + weight_2)

    # return sends the listed values back to the point where this function was called
    return prevalence_1, prevalence_2, duckDistribution_1, duckDistribution_2

def create_sliders(rate_1, rate_2, magnitude_1, magnitude_2, switchiness, timeSteps, summed_magnitude_1, summed_magnitude_2, timeSteps_1, timeSteps_2):
    """Create sliders for the parameters."""
    
    # define custom layout; make it wide enough so slider labels are visible
    custom_layout = Layout(width='1000px')
    style = {'description_width': 'initial'}  # Set the description width to its content


    sliders = [
        IntSlider(min=1, max=10, step=1, value=rate_1, description='Rate 1:'),
        IntSlider(min=1, max=10, step=1, value=rate_2, description='Rate 2:'),
        IntSlider(min=1, max=20, step=1, value=magnitude_1, description='Magnitude 1:'),
        IntSlider(min=1, max=20, step=1, value=magnitude_2, description='Magnitude 2:'),
        FloatSlider(min=0, max=1, step=0.01, value=switchiness, description='Switchiness:'),
        IntSlider(min=10, max=1000, step=1, value=timeSteps, description='Time Steps:'),
        IntSlider(min=1, max=1000, step=1, value=summed_magnitude_1, description='Base mag 1:'),
        IntSlider(min=1, max=1000, step=1, value=summed_magnitude_2, description='Base mag 2:'),
        IntSlider(min=1, max=1000, step=1, value=timeSteps_1, description='Base time 1:'),
        IntSlider(min=1, max=1000, step=1, value=timeSteps_2, description='Base time 2:')
        # if slider labels get cut off, can use this style=style at the end to make wider. 
        # aesthetically more pleasing to just make the text short enough to fit
        #FloatSlider(min=1, max=1000, step=1, value=timeSteps_2, description='Base time 2:', style=style)
    ]
    return sliders

#  (it had to be defined and executed after you called the plot updating 
# function, so I put the definition for reset_values() and calling reset_button after 
# defining update_plot and before creating the interactive widget).
 
def main():
    """
        The *main* function is what will be executed by default given the last line 
        (take a look)
    """
    
    # Define initial [and default] values for the parameters
    rate_1 = 2
    rate_2 = 4 
    magnitude_1 = 10
    magnitude_2 = 10
    switchiness = 0.5
    timeSteps = 100
    summed_magnitude_1 = 100
    summed_magnitude_2 = 100
    timeSteps_1 = 100
    timeSteps_2 = 100

    slider_defaults = [rate_1, rate_2, magnitude_1, magnitude_2, switchiness, timeSteps, 
                       summed_magnitude_1, summed_magnitude_2, timeSteps_1, timeSteps_2]
    
    # Create the sliders
    sliders = create_sliders(*slider_defaults)
    #sliders = create_sliders(rate_1, rate_2, magnitude_1, magnitude_2, switchiness, timeSteps, summed_magnitude_1, summed_magnitude_2, timeSteps_1, timeSteps_2)

    # Create the "Reset" button and set its on_click event to reset_values function
    reset_button = Button(description="Reset")
    reset_button.on_click(lambda b: reset_values(b, sliders, slider_defaults))


    
    # Initialize the graphs
    layout = go.Layout(autosize=False, width=700, height=500, font=dict(size=16), showlegend=True)

    # GRAPH 1
    graph1 = go.FigureWidget(layout=layout)
    graph1.add_trace(go.Scatter(name="Source 1"))
    graph1.add_trace(go.Scatter(name="Source 2"))
#    graph1.update_layout(title_text="Prevalence", xaxis_title="Time Step", yaxis_title="Prevalence", legend=dict(x=0.8, y=0.9, font=dict(size=14)))
    graph1.update_layout(title_text="Prevalence", xaxis_title="Time Step", yaxis_title="Prevalence", legend=dict(font=dict(size=14, color="black")))

    # GRAPH 2
    graph2 = go.FigureWidget(layout=layout)
    graph2.add_trace(go.Scatter(name="Source 1"))
    graph2.add_trace(go.Scatter(name="Source 2"))
    graph2.update_layout(title_text="Duck Distribution", xaxis_title="Time Step", yaxis_title="Duck Distribution")

    # LABELS WILL DISPLAY UPDATED VALUES
    expected_ratio_label = Label(value="Expected ratio: ")
    actual_ratio_label = Label(value="Actual ratio: ")
    discrepancy_label = Label(value="Discrepancy: ")
    duck_proportion_label = Label(value="Duck proportion at source 1: ")
    advice_label = Label(value="\n --- click the Play control for this cell to reset ---")
    magnitude_1_label = Label(value="Current Mag1: ")
    magnitude_2_label = Label(value="Current Mag2: ")
    prevalence_1_label = Label(value="Current Prev1: ")
    prevalence_2_label = Label(value="Current Prev2: ")
    duckDistribution_1_label = Label(value="Current DuckDist1: ")
    duckDistribution_2_label = Label(value="Current DuckDist2: ")


    # Function to update the plots
    def update_plot(rate_1, rate_2, magnitude_1, magnitude_2, switchiness, timeSteps, summed_magnitude_1, summed_magnitude_2, timeSteps_1, timeSteps_2):
        prevalence_1, prevalence_2, duckDistribution_1, duckDistribution_2 = \
            simulation(switchiness, rate_1, rate_2, magnitude_1, magnitude_2, summed_magnitude_1, summed_magnitude_2, timeSteps_1, timeSteps_2, int(timeSteps))
        x_values = np.arange(int(timeSteps))

        # calculate the theoretically expected ratio
        expected_ratio = (magnitude_1/rate_1) / ((magnitude_1/rate_1) + (magnitude_2/rate_2))
        
        # calcuate the actual ratio...
        actual_ratio = duckDistribution_1[-1]
        
        # and the difference between expected and actual
        discrepancy = abs(actual_ratio - expected_ratio)

        # graph update instrucions
        with graph1.batch_update():
            graph1.data[0].x = x_values
            graph1.data[0].y = prevalence_1
            graph1.data[1].x = x_values
            graph1.data[1].y = prevalence_2
            showlegend=True

        with graph2.batch_update():
            graph2.data[0].x = x_values
            graph2.data[0].y = duckDistribution_1
            graph2.data[1].x = x_values
            graph2.data[1].y = duckDistribution_2

        # print 'labels'    
        expected_ratio_label.value = "Expected ratio: {:.3f}".format(expected_ratio)
        actual_ratio_label.value = "Actual ratio:  {:.3f}".format(actual_ratio)
        discrepancy_label.value = "Discrepancy:  {:.3f}".format(discrepancy)
        duck_proportion_label.value = "Duck proportion at source 1: {:.3f}".format(duckDistribution_1[-1])
        magnitude_1_label.value = f"CurMag1: {magnitude_1}"
        magnitude_2_label.value = f"CurMag2: {magnitude_2}"
        prevalence_1_label.value = f"CurPrev1: {prevalence_1[-1]:.3f}"
        prevalence_2_label.value = f"CurPrev2: {prevalence_2[-1]:.3f}"
        duckDistribution_1_label.value = f"CurDuckDis1: {duckDistribution_1[-1]:.3f}"
        duckDistribution_2_label.value = f"CurDuckDis2: {duckDistribution_2[-1]:.3f}"
        advice_label.value = "\n ### click the Play control for this cell to reset ###"
        
    
    def reset_values(button, sliders, defaults):
        """Reset the interactive widgets to their default values."""
        for slider, default in zip(sliders, defaults):
            slider.value = default
        # Force the update
        update_plot(*[slider.value for slider in sliders])
        # Create the interactive widget
   
 
    # Create the interactive widget
    widget = interactive(update_plot, 
                         rate_1=sliders[0], 
                         rate_2=sliders[1], 
                         magnitude_1=sliders[2], 
                         magnitude_2=sliders[3], 
                         switchiness=sliders[4], 
                         timeSteps=sliders[5], 
                         summed_magnitude_1=sliders[6], 
                         summed_magnitude_2=sliders[7], 
                         timeSteps_1=sliders[8], 
                         timeSteps_2=sliders[9])

    # Display the sliders, labels and plots
    #display(VBox([widget, HBox([expected_ratio_label, actual_ratio_label, discrepancy_label, advice_label]), HBox([graph1, graph2])]))
    display(reset_button)
    display(VBox([widget, 
              HBox([expected_ratio_label, actual_ratio_label, discrepancy_label, advice_label]), 
              HBox([magnitude_1_label, magnitude_2_label, prevalence_1_label, prevalence_2_label, duckDistribution_1_label, duckDistribution_2_label]),
              HBox([graph1, graph2])]))


# Run the main function


main()


Button(description='Reset', style=ButtonStyle())

## LAB REPORT

`Answer the questions below. It works best to copy and paste the questions into your report and insert your answers below each prompt. Remember, submit a 'static' file (.txt, .rtf, .doc, .pdf, etc.) and not a dynamic document link (e.g., to google docs). Make sure to include your name, date, and topic at the top of your report ('Math models of probability matching').`

#### Required questions

1. Explore a few parameter combinations and other controls. 
   1. With the defaults, how large is the discrepancy?
   2. Try increasing the main timeSteps parameter; does the discrepancy ever approach zero? *(Note that you can use the timeSteps slider to extend or shorten the plots.*
   3. Is there any parameter you think could help? Which one? Try it and report whether it helps.
   4. If the parameter you chose did not help, try some others. Make sure to reset between each try (by clicking the play icon for the cell). *Hint: there is definitely a parameter that will work!*
   

2. Once you have found parameter settings that allow a low discrepancy with all the other original parameters, try some new combinations of rates and magnitudes. To keep things simple, I recommend keeping either the two rates or the two magnitudes the same, and then only varying one of them (e.g., set rate_1 and rate_2 to 2, but set magnitude_1 and magnitude_2 to different values. Try at least 2 additional rate differences and 2 magnitude differences. 
   1. In general, does the system tend to be able to predict probability matching for differences in rate? In magnitude?
   2. For the special parameter that maximizes fit (i.e., minimizes discrepancy [makes it as small as possible]), do you need a new value for every rate or magnitude combination? Or is there a value that yields fairly low discrepancy for most combinations?
   

3. What happens when you change the *base_magnitude* and *base_time* parameters? *Tip -- it usually works best to keep the two base_time values the same (e.g., both 100 or both 200) and manipulate base_size values instead.*
   
#### Challenge questions


4. Imagine we actually went out to a UConn pond and did these kinds of experiments. 
   1. What data would we collect?
   2. How would we compare the data to the model predictions? 
   3. How closely would the data and model have to correspond for us to conclude the model is correct or at least not completely wrong? Remember, we want to generate testable predictions from the model 
   4. Can you think of aspects of the real-world situation that are not captured in the model? *Think about extreme values; does the model make absurd predictions -- e.g., what if one source is throwing 100 kilograms of bread every time step, and the other is throwing 50 kg per time step...? Beyond such cases that we could argue should be outside the scope of the model, can you think of other ways the model might go wrong?* 
   5. Could you use any of these possibilities to falsify the model? 
   

5. Try to do one of these, but if you get stuck, just answer "got stuck". 
   1. *Modeling challenge:* Can you think of any assumptions in the model you think could be different? Download the notebook and try to modify the code to make the change you have in mind. Report whether/how your revised model performs differently from the original. 
   2. *Pure programming challenge:* I tried to create a reset button that you could click and reinitilize everything. I ran out of time and did not get it running. You can see traces of what I tried in the code... See if you can get it to work. Again, download the notebook so you can make changes locally (or on colab). You'll need to add the reset_button to the display call. 
   